# Module 6: Python 메모리 관리와 가비지 컬렉션

이 노트북에서는 Python의 메모리 관리 메커니즘과 가비지 컬렉션(Garbage Collection)에 대해 학습합니다.

## 학습 목표
- Python의 참조 카운팅 메커니즘 이해
- 순환 참조와 가비지 컬렉션의 동작 원리 파악
- `gc` 모듈을 활용한 디버깅 기법 습득
- `tracemalloc`을 이용한 메모리 프로파일링
- `weakref`를 활용한 메모리 누수 방지 패턴

## 1. 소개: Python 메모리 관리

Python은 메모리 관리를 위해 두 가지 주요 메커니즘을 사용합니다:

### 1.1 참조 카운팅 (Reference Counting)
- Python의 주요 메모리 관리 방식
- 각 객체는 참조 횟수를 추적하는 카운터를 가짐
- 참조 카운트가 0이 되면 즉시 메모리 해제
- **장점**: 즉각적인 메모리 해제, 예측 가능한 동작
- **단점**: 순환 참조 감지 불가

### 1.2 가비지 컬렉션 (Garbage Collection)
- 순환 참조를 감지하고 해결하기 위한 보조 메커니즘
- 세대(generation) 기반 수집 전략 사용
- 주기적으로 순환 참조 그룹을 찾아 메모리 해제

```
┌─────────────────────────────────────────────────────┐
│              Python Memory Management               │
├─────────────────────────────────────────────────────┤
│  ┌─────────────────┐    ┌───────────────────────┐  │
│  │ Reference Count │───▶│ Immediate Deallocation│  │
│  └─────────────────┘    └───────────────────────┘  │
│           │                                        │
│           ▼ (circular ref)                         │
│  ┌─────────────────┐    ┌───────────────────────┐  │
│  │   GC (Cycle)    │───▶│  Generational Collection │
│  └─────────────────┘    └───────────────────────┘  │
└─────────────────────────────────────────────────────┘
```

## 2. 참조 카운팅 (Reference Counting)

### 2.1 sys.getrefcount() 사용하기

Python의 `sys` 모듈은 객체의 참조 카운트를 확인하는 `getrefcount()` 함수를 제공합니다.

**주의**: `getrefcount()`를 호출할 때 인자로 전달되는 객체는 함수 난 내 임시 참조가 생성되므로,
반환값은 실제 참조 카운트보다 1이 더 높게 표시됩니다.

In [ ]:
import sys

# 리스트 객체 생성
a = [1, 2, 3]

# 초기 참조 카운트 확인
# 참고: getrefcount() 호출 시 임시 참조가 생성되므로 결과는 실제보다 1 높음
print(f"Initial refcount: {sys.getrefcount(a)}")

# b가 a를 참조
b = a
print(f"After b = a: {sys.getrefcount(a)}")

# c도 a를 참조
c = a
print(f"After c = a: {sys.getrefcount(a)}")

# 참조 제거
del b
print(f"After del b: {sys.getrefcount(a)}")

del c
print(f"After del c: {sys.getrefcount(a)}")

### 2.2 다양한 데이터 타입의 참조 카운트

불변 객체(immutable)와 가변 객체(mutable)의 참조 카운트 동작을 비교해 봅시다.

In [ ]:
import sys

# 불변 객체: 작은 정수는 interning되어 재사용됨
x = 100
y = 100
print(f"Small int (100) - x: {sys.getrefcount(x)}, y: {sys.getrefcount(y)}")
print(f"x is y: {x is y}")

# 큰 정수는 별도 객체로 생성
big_x = 1000000
big_y = 1000000
print(f"\nBig int (1000000) - big_x: {sys.getrefcount(big_x)}, big_y: {sys.getrefcount(big_y)}")
print(f"big_x is big_y: {big_x is big_y}")

# 문자염 interning
s1 = "hello"
s2 = "hello"
print(f"\nString 'hello' - s1: {sys.getrefcount(s1)}, s2: {sys.getrefcount(s2)}")
print(f"s1 is s2: {s1 is s2}")

# 가변 객체: 리스트는 항상 새로운 객체
list1 = [1, 2, 3]
list2 = [1, 2, 3]
print(f"\nList [1,2,3] - list1: {sys.getrefcount(list1)}, list2: {sys.getrefcount(list2)}")
print(f"list1 is list2: {list1 is list2}")

### 2.3 함수 호출에서의 참조 카운트

함수에 객체를 전달할 때 참조 카운트가 어떻게 변하는지 확인합니다.

In [ ]:
import sys

def check_refcount(obj, label="inside function"):
    """함수 내부에서 참조 카운트 확인"""
    print(f"{label}: {sys.getrefcount(obj)}")

# 테스트 객체
data = {"key": "value"}
print(f"Before function call: {sys.getrefcount(data)}")

# 함수 호출 - 인자로 전달되면서 참조 카운트 증가
check_refcount(data)

print(f"After function call: {sys.getrefcount(data)}")

# 함수 내부에서 추가 참조
def store_reference(obj):
    stored = obj  # 지역 변수로 참조
    print(f"With local variable: {sys.getrefcount(obj)}")
    return stored

result = store_reference(data)
print(f"After store_reference: {sys.getrefcount(data)}")
del result
print(f"After del result: {sys.getrefcount(data)}")

## 3. 순환 참조 (Circular Reference)

순환 참조는 두 개 이상의 객체가 서로를 참조하여 참조 카운트가 0이 되지 않는 상황입니다.
이는 참조 카운팅만으로는 메모리를 해제할 수 없는 문제를 야기합니다.

In [ ]:
import sys

class Node:
    """순환 참조를 만들기 위한 간단한 노드 클래스"""
    def __init__(self, name):
        self.name = name
        self.ref = None
    
    def __repr__(self):
        return f"Node({self.name})"
    
    def __del__(self):
        """소멸자 - 객체가 삭제될 때 호출"""
        print(f"  [__del__] {self.name} is being destroyed")

# 순환 참조 생성
print("=== Creating circular reference ===")
node_a = Node("A")
node_b = Node("B")

print(f"Initial: node_a refcount = {sys.getrefcount(node_a) - 1}")
print(f"Initial: node_b refcount = {sys.getrefcount(node_b) - 1}")

# 순환 참조 설정
node_a.ref = node_b
node_b.ref = node_a

print(f"\nAfter circular ref: node_a refcount = {sys.getrefcount(node_a) - 1}")
print(f"After circular ref: node_b refcount = {sys.getrefcount(node_b) - 1}")

print("\n=== Deleting external references ===")
del node_a
del node_b

print("External references deleted, but circular reference remains...")
print("(GC가 실행되기 전까지 메모리에 남아있음)")

### 3.1 순환 참조 시각화

```
Before del:                    After del (circular ref remains):
                               
  ┌─────────┐                   ┌─────────┐
  │ node_a  │───▶┌───────┐      │   ???   │   (외부 참조 없음)
  └─────────┘    │ Node  │      └─────────┘
                 │  "A"  │◄──────┐    ┌───┐
                 └───┬───┘       │    │   │
                     │ ref       │    ▼   │
                     ▼           │  ┌─────┴───┐
                 ┌───────┐       └──┤  Node   │
                 │ Node  │◄─────────│   "B"   │
                 │  "B"  │    ref   └─────┬───┘
                 └───┬───┘                │
                     │ ref                │
                     ▼                    │
  ┌─────────┐    ┌───────┐               │
  │ node_b  │───▶│       │◄──────────────┘
  └─────────┘    └───────┘

  참조 카운트: A=2, B=2          참조 카운트: A=1, B=1 (0이 아니므로 메모리 유지)
```

## 4. gc 모듈 - 가비지 컬렉터 제어

Python의 `gc` 모듈은 가비지 컬렉터를 제어하고 디버깅하는 기능을 제공합니다.

In [ ]:
import gc

# GC 상태 확인
print("=== GC Status ===")
print(f"GC enabled: {gc.isenabled()}")
print(f"GC thresholds: {gc.get_threshold()}")
print(f"Current counts: {gc.get_count()}")

### 4.1 디버그 모드 활성화

GC의 동작을 자세히 관찰하기 위해 디버그 모드를 활성화합니다.

In [ ]:
import gc

# 디버그 플래그 설정
# DEBUG_STATS: 수집 통계 출력
# DEBUG_COLLECTABLE: 수집 가능한 객체 출력
# DEBUG_UNCOLLECTABLE: 수집 불가능한 객체 출력
# DEBUG_SAVEALL: 삭제된 객체를 gc.garbage에 저장

gc.set_debug(gc.DEBUG_STATS | gc.DEBUG_COLLECTABLE)

class TrackedNode:
    def __init__(self, name):
        self.name = name
        self.ref = None
    
    def __repr__(self):
        return f"TrackedNode({self.name})"

print("=== Creating circular reference ===")
a = TrackedNode("A")
b = TrackedNode("B")
a.ref = b
b.ref = a

# 외부 참조 제거
print("\n=== Removing external references ===")
del a
del b

# 수동으로 GC 실행
print("\n=== Running garbage collection ===")
collected = gc.collect()
print(f"\nCollected objects: {collected}")

# 디버그 모드 해제
gc.set_debug(0)

### 4.2 세대별 가비지 컬렉션

Python의 GC는 세 가지 세대(generation)로 객체를 관리합니다:

- **Gen 0**: 새로 생성된 객체
- **Gen 1**: Gen 0에서 살아남은 객체
- **Gen 2**: Gen 1에서 살아남은 객체 (가장 오래된 세대)

객체가 GC에서 살아남을수록 더 높은 세대로 이동하며, 높은 세대는 더 적은 빈도로 검사됩니다.

In [ ]:
import gc

# GC 임계값 확인
print("=== GC Thresholds ===")
print("Thresholds: (gen0, gen1, gen2)")
print(f"  Current: {gc.get_threshold()}")
print("\nMeaning:")
print("  - Gen 0: 할당 - 해제가 700번 차이나면 수집")
print("  - Gen 1: Gen 0 수집이 10번 실행되면 수집")
print("  - Gen 2: Gen 1 수집이 10번 실행되면 수집")

# 현재 세대별 객체 수
print(f"\nCurrent object counts: {gc.get_count()}")

In [ ]:
import gc

class GenTracker:
    """세대 추적을 위한 클래스"""
    def __init__(self, name):
        self.name = name
        self.ref = None
    
    def __repr__(self):
        return f"GenTracker({self.name})"

def get_gen_counts():
    """각 세대의 객체 수 반환"""
    return gc.get_count()

# 초기 상태
print("Initial state:")
print(f"  Counts: {get_gen_counts()}")

# 객체 생성
print("\n=== Creating objects ===")
objects = []
for i in range(5):
    obj = GenTracker(f"obj_{i}")
    objects.append(obj)
    print(f"After creating {obj}: {get_gen_counts()}")

# 객체 목록 비우기 (참조 제거)
print("\n=== Clearing references ===")
objects.clear()
print(f"After clear: {get_gen_counts()}")

# 특정 세대만 수집
print("\n=== Collecting specific generations ===")

print("\nCollecting Gen 0 only...")
collected = gc.collect(0)
print(f"  Collected: {collected}, Counts: {get_gen_counts()}")

print("\nCollecting Gen 1...")
collected = gc.collect(1)
print(f"  Collected: {collected}, Counts: {get_gen_counts()}")

print("\nCollecting all generations...")
collected = gc.collect(2)  # 또는 gc.collect()
print(f"  Collected: {collected}, Counts: {get_gen_counts()}")

### 4.3 수집 불가능한 객체 확인

`gc.garbage` 리스트는 수집 불가능한 객체를 저장합니다.
(DEBUG_SAVEALL 플래그가 설정된 경우)

In [ ]:
import gc

# 수집된 객체 저장 모드 활성화
gc.set_debug(gc.DEBUG_SAVEALL)

class Uncollectable:
    def __init__(self):
        self.ref = self  # 자기 자신을 참조 (순환 참조)
    
    def __del__(self):
        print("  __del__ called")
    
    def __repr__(self):
        return "Uncollectable()"

# 순환 참조 객체 생성
print("Creating self-referencing object...")
u = Uncollectable()
del u

# GC 실행
print("\nRunning GC...")
gc.collect()

# gc.garbage 확인
print(f"\nObjects in gc.garbage: {len(gc.garbage)}")
for obj in gc.garbage:
    print(f"  - {obj}")

# 정리
gc.garbage.clear()
gc.set_debug(0)

## 5. tracemalloc으로 메모리 프로파일링

`tracemalloc` 모듈은 Python 3.4+에서 제공하는 메모리 추적 도구입니다.
메모리 사용량을 측정하고 누수를 찾는 데 유용합니다.

In [ ]:
import tracemalloc

# 메모리 추적 시작
tracemalloc.start()

# 현재 메모리 사용량 확인
current, peak = tracemalloc.get_traced_memory()
print(f"Initial memory usage:")
print(f"  Current: {current / 1024:.2f} KB")
print(f"  Peak: {peak / 1024:.2f} KB")

In [ ]:
import tracemalloc

# 첫 번째 스냅샷
snapshot1 = tracemalloc.take_snapshot()

# 메모리를 사용하는 작업 수행
print("Creating large data structures...")
large_list = [i for i in range(100000)]
large_dict = {f"key_{i}": i for i in range(10000)}

# 두 번째 스냅샷
snapshot2 = tracemalloc.take_snapshot()

# 메모리 사용량 변화
current, peak = tracemalloc.get_traced_memory()
print(f"\nAfter creating objects:")
print(f"  Current: {current / 1024:.2f} KB")
print(f"  Peak: {peak / 1024:.2f} KB")

In [ ]:
# 스냅샷 비교
top_stats = snapshot2.compare_to(snapshot1, 'lineno')

print("=== Top 10 Memory Differences ===")
for stat in top_stats[:10]:
    print(f"{stat}")

In [ ]:
# 가장 많은 메모리를 사용하는 코드 라인 확인
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory: {current / 1024:.2f} KB")

# 현재 메모리 상위 10개
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("\n=== Top 10 Memory Consumers ===")
for stat in top_stats[:10]:
    print(f"{stat}")

# 메모리 추적 중지
tracemalloc.stop()
print("\nTracemalloc stopped.")

### 5.1 메모리 누수 감지 예제

실제 메모리 누수 패턴을 감지하는 예제입니다.

In [ ]:
import tracemalloc

class MemoryLeak:
    """의도적으로 메모리 누수를 발생시키는 클래스"""
    _cache = []  # 클래스 레벨 캐시 (계속 누적됨)
    
    def __init__(self, data):
        self.data = data
        # 문제: 이 객체가 _cache에 계속 쌓임
        MemoryLeak._cache.append(self)

# 메모리 추적 시작
tracemalloc.start()

# 초기 스냅샷
snapshot1 = tracemalloc.take_snapshot()

# 메모리 누수 시뮬레이션
print("Simulating memory leak...")
for i in range(100):
    _ = MemoryLeak([i] * 1000)  # 큰 객체 생성

# 두 번째 스냅샷
snapshot2 = tracemalloc.take_snapshot()

# 비교
diff = snapshot2.compare_to(snapshot1, 'lineno')
print("\n=== Memory Leak Detection ===")
for stat in diff[:5]:
    if stat.size_diff > 0:
        print(f"+{stat.size_diff / 1024:.2f} KB: {stat.traceback.format()[-1]}")

# 정리
MemoryLeak._cache.clear()
tracemalloc.stop()

## 6. weakref - 약한 참조로 순환 참조 방지

`weakref` 모듈은 객체를 참조하지만 참조 카운트를 증가시키지 않는 '약한 참조'를 제공합니다.
이를 통해 순환 참조 없이 객체 간의 관계를 표현할 수 있습니다.

In [ ]:
import weakref
import sys

class Node:
    """약한 참조를 사용하는 노드"""
    def __init__(self, name):
        self.name = name
        self._ref = None
    
    @property
    def ref(self):
        """약한 참조를 통해 객체 반환"""
        if self._ref is None:
            return None
        return self._ref()  # 약한 참조는 callable로 객체 얻음
    
    @ref.setter
    def ref(self, value):
        """약한 참조 설정"""
        if value is None:
            self._ref = None
        else:
            self._ref = weakref.ref(value)
    
    def __repr__(self):
        return f"Node({self.name})"
    
    def __del__(self):
        print(f"  [__del__] {self.name} destroyed")

print("=== Using weakref to prevent circular reference ===")

node_a = Node("A")
node_b = Node("B")

print(f"Before linking:")
print(f"  node_a refcount: {sys.getrefcount(node_a) - 1}")
print(f"  node_b refcount: {sys.getrefcount(node_b) - 1}")

# 약한 참조로 연결
node_a.ref = node_b
node_b.ref = node_a

print(f"\nAfter weak linking:")
print(f"  node_a refcount: {sys.getrefcount(node_a) - 1}")
print(f"  node_b refcount: {sys.getrefcount(node_b) - 1}")

print(f"\nAccessing via weakref: {node_a.ref}")

print("\n=== Deleting nodes ===")
del node_a
del node_b
print("Nodes deleted successfully (no circular reference!)")

### 6.1 weakref.ref와 weakref.proxy

`weakref`는 두 가지 주요 타입을 제공합니다:
- `weakref.ref()`: 객체에 대한 약한 참조를 생성, `()`로 접근
- `weakref.proxy()`: 객체의 프록시를 생성, 직접 속성 접근 가능

In [ ]:
import weakref

class DataObject:
    def __init__(self, value):
        self.value = value
    
    def process(self):
        return self.value * 2

# 원본 객체
obj = DataObject(42)

# weakref.ref 사용
ref = weakref.ref(obj)
print(f"weakref.ref: {ref}")
print(f"Accessing: {ref()}")
print(f"Value: {ref().value}")

# weakref.proxy 사용
proxy = weakref.proxy(obj)
print(f"\nweakref.proxy: {proxy}")
print(f"Direct access: {proxy.value}")
print(f"Method call: {proxy.process()}")

# 원본 삭제 후
print("\n=== After deleting original ===")
del obj

# ref는 None 반환
print(f"ref() is None: {ref() is None}")

# proxy는 ReferenceError 발생
try:
    print(proxy.value)
except ReferenceError as e:
    print(f"ReferenceError: {e}")

### 6.2 WeakKeyDictionary와 WeakValueDictionary

`weakref`는 키나 값이 약하게 참조되는 딕셔너리 구현도 제공합니다.

In [ ]:
import weakref

# WeakValueDictionary: 값이 약하게 참조됨
# 값이 더 이상 강한 참조가 없으면 자동으로 딕셔너리에서 제거

class Cacheable:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return f"Cacheable({self.name})"

# 캐시 생성
cache = weakref.WeakValueDictionary()

# 객체 생성 및 캐싱
obj1 = Cacheable("object1")
obj2 = Cacheable("object2")

cache['key1'] = obj1
cache['key2'] = obj2

print(f"Cache contents: {dict(cache)}")

# obj1에 대한 강한 참조 제거
print("\nDeleting obj1...")
del obj1

# GC 실행하여 약한 참조 정리
import gc
gc.collect()

print(f"Cache after del obj1: {dict(cache)}")

# obj2는 여전히 존재
print(f"obj2 still accessible: {cache.get('key2')}")

## 7. 종합 예제: 메모리 관리 모범 사례

실제 애플리케이션에서 메모리 관리를 위한 모범 사례를 살펴봅시다.

In [ ]:
import weakref
import gc
from contextlib import contextmanager

class Observable:
    """옵서버 패턴 - 약한 참조로 메모리 누수 방지"""
    def __init__(self):
        # 약한 참조를 사용하여 옵서버 등록
        self._observers = weakref.WeakSet()
    
    def add_observer(self, observer):
        """옵서버 등록"""
        self._observers.add(observer)
        print(f"Observer added: {observer}")
    
    def notify(self, event):
        """모든 옵서버에게 알림"""
        dead_observers = []
        for observer in self._observers:
            try:
                observer.on_event(event)
            except Exception as e:
                print(f"Error notifying {observer}: {e}")
    
    def observer_count(self):
        return len(self._observers)

class Observer:
    def __init__(self, name):
        self.name = name
    
    def on_event(self, event):
        print(f"  [{self.name}] Received: {event}")
    
    def __repr__(self):
        return f"Observer({self.name})"
    
    def __del__(self):
        print(f"  [__del__] {self.name} destroyed")

# 사용 예제
print("=== Observer Pattern with weakref ===")

observable = Observable()

# 옵서버 생성 및 등록
obs1 = Observer("Observer1")
obs2 = Observer("Observer2")

observable.add_observer(obs1)
observable.add_observer(obs2)

print(f"\nObserver count: {observable.observer_count()}")

# 이벤트 알림
print("\nNotifying observers...")
observable.notify("Hello!")

# 옵서버 중 하나 삭제
print("\nDeleting obs1...")
del obs1
gc.collect()

print(f"Observer count after del: {observable.observer_count()}")

print("\nNotifying remaining observers...")
observable.notify("Still there?")

## 8. 연습 문제

다음 문제들을 직접 해결해보며 학습한 내용을 실습하세요.

### 문제 1: 참조 카운트 추적

다음 코드의 각 단계에서 `data` 객체의 참조 카운트가 어떻게 변하는지 예측하고 확인하세요.

In [ ]:
import sys

def track_refcount():
    data = {"key": "value"}
    
    # 여기에 각 단계별 참조 카운트 출력 코드 작성
    # TODO: 1. 초기 참조 카운트
    
    ref1 = data
    # TODO: 2. ref1 할당 후
    
    ref2 = ref1
    # TODO: 3. ref2 할당 후
    
    del ref1
    # TODO: 4. ref1 삭제 후
    
    return data

result = track_refcount()
# TODO: 5. 함수 반환 후

### 문제 2: 순환 참조 수정하기

다음 코드에서 순환 참조를 `weakref`를 사용하여 제거하세요.

In [ ]:
# 원본 코드 (순환 참조 문제)
class Parent:
    def __init__(self, name):
        self.name = name
        self.children = []
    
    def add_child(self, child):
        self.children.append(child)
        child.parent = self  # 순환 참조 발생!

class Child:
    def __init__(self, name):
        self.name = name
        self.parent = None
    
    def __del__(self):
        print(f"Child {self.name} destroyed")

# TODO: weakref를 사용하여 순환 참조 제거
# 힌트: child.parent를 약한 참조로 변경

### 문제 3: 메모리 프로파일링 실습

다음 함수의 메모리 사용량을 `tracemalloc`으로 측정하고,
가장 많은 메모리를 사용하는 코드 라인을 찾으세요.

In [ ]:
def process_data(n):
    """메모리 사용량을 측정할 함수"""
    # 리스트 생성
    data = []
    for i in range(n):
        data.append([i] * 100)
    
    # 딕셔너리 생성
    lookup = {}
    for i, item in enumerate(data):
        lookup[f"item_{i}"] = item
    
    # 처리
    result = []
    for key in lookup:
        result.append(sum(lookup[key]))
    
    return result

# TODO: tracemalloc을 사용하여 메모리 프로파일링
# 1. tracemalloc.start() 호출
# 2. 함수 실행 전 스냅샷
# 3. process_data(1000) 실행
# 4. 함수 실행 후 스냅샷
# 5. 스냅샷 비교하여 메모리 사용량 분석

### 문제 4: GC 임계값 조정

GC 임계값을 조정하여 메모리 사용량과 성능의 트레이드오프를 실험해보세요.

In [ ]:
import gc
import time

def create_circular_refs(count):
    """순환 참조 객체 생성"""
    class Node:
        def __init__(self):
            self.ref = None
    
    nodes = [Node() for _ in range(count)]
    for i in range(count):
        nodes[i].ref = nodes[(i + 1) % count]
    
    return nodes

# TODO: GC 임계값 실험
# 1. 기본 임계값 (700, 10, 10)에서 실행 시간 측정
# 2. 임계값을 (100, 5, 5)로 낮추고 실행 시간 측정
# 3. 임계값을 (2000, 20, 20)으로 높이고 실행 시간 측정
# 4. 각 설정에서 메모리 사용량도 함께 비교

print("Experiment with different GC thresholds")
print("Hint: Use gc.set_threshold() and time.time()")

## 9. 요약

이 노트북에서 학습한 내용:

### 핵심 개념
1. **참조 카운팅**: Python의 주요 메모리 관리 방식
   - `sys.getrefcount()`로 참조 카운트 확인
   - 참조 카운트가 0이 되면 즉시 메모리 해제

2. **순환 참조**: 객체들이 서로를 참조하여 참조 카운트가 0이 되지 않는 문제
   - 참조 카운팅만으로는 해결 불가
   - 가비지 컬렉터가 순환 참조를 감지하고 해제

3. **가비지 컬렉션**: 세대 기반 순환 참조 수집
   - `gc` 모듈로 제어 및 디버깅 가능
   - Gen 0, 1, 2 세대로 객체 관리

4. **메모리 프로파일링**: `tracemalloc`으로 메모리 사용량 분석
   - 스냅샷 비교로 메모리 누수 감지
   - 코드 라인별 메모리 사용량 확인

5. **약한 참조**: `weakref`로 순환 참조 방지
   - 참조 카운트를 증가시키지 않는 참조
   - 캐시, 옵서버 패턴 등에 활용

### 모범 사례
- 순환 참조가 발생할 수 있는 구조에서는 `weakref` 사용 고려
- 메모리 누수 의심 시 `tracemalloc`으로 프로파일링
- `gc` 모듈 디버그 모드로 GC 동작 관찰
- `__del__` 소멸자 사용 시 순환 참조 주의